<a href="https://colab.research.google.com/github/pk2971/Capstone/blob/main/Code/Text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import re
file_path = '/content/1929.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()


In [47]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import Dataset  # Import the Dataset class
from tqdm import tqdm

In [48]:
keywords_to_match = ['she', 'her', 'hers', 'lady', 'ladies', 'woman', 'women', 'girl', 'girls']

In [49]:
def extract_sentences_with_keywords(text, keywords):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

    filtered_sentences = [
        sentence for sentence in sentences
        if any(re.search(rf'\b{re.escape(keyword)}\b', sentence, re.IGNORECASE) for keyword in keywords)
    ]

    return '\n'.join(filtered_sentences)


In [50]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-large-cnn"  # You can choose a different BART variant
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)


filtered_sentences=extract_sentences_with_keywords(content, keywords_to_match)

paragraphs = re.split(r'\n\n', filtered_sentences)

unique_paragraphs = set()

all_summaries=""
# Iterate through the paragraphs and add them to the set
for paragraph in paragraphs:
    # Remove leading and trailing whitespaces for better comparison
    cleaned_paragraph = paragraph.strip()

    # Check if the paragraph is not empty
    if cleaned_paragraph:
        unique_paragraphs.add(cleaned_paragraph)

for part in unique_paragraphs:

  input_ids = tokenizer.encode(part, return_tensors="pt", max_length=1024, truncation=True)
  summary_ids = model.generate(input_ids, max_length=200, num_beams=5, length_penalty=2.5,num_return_sequences=3, early_stopping=True)

  summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

  all_summaries += f"{summary}\n\n"

  print("Generated Summary:", summary)



Generated Summary: The full amount of an endowment policy will not be paid if the insured person dies before the policy matures. No separate records are ordinarily kept of persons who come from Ireland to Great Britain. The number of insured workpeople registered as unemployed on the register of the Walworth Road (Borough) Employment Exchange on February.
Generated Summary: asked the Minister of Transport whether his attention has been drawn to the number of outrages committed upon women in non-corridor trains. Asked the First Lord of the Admiralty how many men have received notice of discharge in the works departments and other departments, Devonport Dockyard.
Generated Summary: I propose to lay a White Paper showing the full financial effect of any new Clause which may be introduced on behalf of the Government to take the place of Clause 4 of the present Unemployment Bill as introduced. When will it be laid, and shall we have ample notice both of the terms of the new Clause and of it

In [53]:
output_file_path = "/content/summaries.txt"
with open(output_file_path, "w", encoding="utf-8") as file:
    file.write(all_summaries)